### Antenneregister

In [29]:
# -q option for suppressing output for less clutter

!pip install pandas -q

In [3]:
import pandas as pd

### This step, loading, is required since data is required in memory, not on disk

Make sure that folder containing antenneregister data is working directory

In [15]:
%pwd
%cd ../data
%pwd

/home/ad/work/data-night-13-bridging-notebooks-and-databases/data


/home/ad/work/data-night-13-bridging-notebooks-and-databases/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/ad/work/data-night-13-bridging-notebooks-and-databases/data'

In [16]:
df = pd.read_csv("antennes-2025-11-10.csv", sep=";")

In [17]:
df.head()

,X,Y,hoofdsoort,SAT_CODE,postcode,woonplaats,gemeente,plaatsing,ingebruikname,OBJECTID,hoogte,hoofdstraalrichting,frequentie,vermogen,veiligeAfstand
0,100006,460068,ZENDAMATEURS,ZENDAMATEUR,2394TR,HAZERSWOUDE-RIJNDIJK,Rijnwoude,NaN,2008-03-03,1.251914e+10,NaN,--,NaN,NaN,-1.00
1,100016,396255,ZENDAMATEURS,ZENDAMATEUR,4711BA,NaN,Rucphen,NaN,2008-03-03,1.436787e+10,NaN,--,NaN,NaN,-1.00
2,100016,436542,"5G NR, GSM, LTE",5G NR,2922AJ,KRIMPEN AAN DEN IJSSEL,Krimpen aan den IJssel,2007-12-18,2020-07-21,3.754851e+10,20.5,240.0,2160 MHz,38.1,9.24
3,100016,436542,"5G NR, GSM, LTE",5G NR,2922AJ,KRIMPEN AAN DEN IJSSEL,Krimpen aan den IJssel,2007-12-18,2020-07-21,4.124185e+10,20.5,120.0,2160 MHz,38.1,9.24
4,100016,436542,"5G NR, GSM, LTE",5G NR,2922AJ,KRIMPEN AAN DEN IJSSEL,Krimpen aan den IJssel,2007-12-18,2020-07-21,5.740544e+10,20.5,0.0,773 MHz,34.0,17.38


### Filtering data: all 5G sites higher than 25m meter in Tilburg, ordered by length (desc)

In [18]:
df[(df["hoogte"] > 25) & (df["gemeente"] == "Tilburg") & (df["SAT_CODE"] == "5G NR")].sort_values(by="hoogte", ascending=False)

,X,Y,hoofdsoort,SAT_CODE,postcode,woonplaats,gemeente,plaatsing,ingebruikname,OBJECTID,hoogte,hoofdstraalrichting,frequentie,vermogen,veiligeAfstand
91615,134187,396299,"5G NR, GSM, LTE",5G NR,5038TC,TILBURG,Tilburg,2012-05-31,2022-12-01,3.517219e+09,47.3,280.0,3700 MHz,49.0,32.41
91619,134187,396299,"5G NR, GSM, LTE",5G NR,5038TC,TILBURG,Tilburg,2012-05-31,2022-12-01,9.165316e+09,47.3,170.0,3700 MHz,49.0,32.41
91618,134187,396299,"5G NR, GSM, LTE",5G NR,5038TC,TILBURG,Tilburg,2012-05-31,2022-12-01,7.782767e+10,47.3,20.0,3700 MHz,49.0,32.41
80162,130220,398774,"LTE, UMTS, 5G NR",5G NR,5044EA,Tilburg,Tilburg,2021-03-12,2021-03-12,2.090666e+10,46.6,240.0,3600 MHz,22.0,3.25
80160,130220,398774,"LTE, UMTS, 5G NR",5G NR,5044EA,Tilburg,Tilburg,2021-03-12,2021-03-12,1.447321e+10,46.6,20.0,783 MHz,22.0,2.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96439,135805,398085,5G NR,5G NR,5015BP,NaN,Tilburg,2024-07-25,2024-08-08,7.336375e+10,25.1,300.0,3500 MHz,46.0,22.94
84089,131958,396886,"5G NR, GSM, LTE",5G NR,5037HC,TILBURG,Tilburg,2024-04-22,2024-05-22,9.637639e+10,25.1,320.0,3700 MHz,48.2,29.56
84083,131958,396886,"5G NR, GSM, LTE",5G NR,5037HC,TILBURG,Tilburg,2024-04-22,2024-05-22,2.106542e+10,25.1,200.0,2160 MHz,37.9,50.16
84081,131958,396886,"5G NR, GSM, LTE",5G NR,5037HC,TILBURG,Tilburg,2024-04-22,2024-05-22,2.062529e+10,25.1,90.0,773 MHz,34.0,9.20


### Display sites on a map

In [30]:
# -q option for suppressing output for less clutter

!pip install folium -q
!pip install pyproj -q

In [20]:
import folium
from pyproj import CRS
from pyproj import Transformer

In [9]:
df_tilburg = df[(df["hoogte"] > 25) & (df["gemeente"] == "Tilburg") & (df["SAT_CODE"] == "5G NR")].sort_values(by="hoogte", ascending=False)

In [10]:
df_coor = df_tilburg[['X', 'Y', 'hoogte']]

### Transform coordinates in order to display on map

In [11]:
transformer = Transformer.from_crs("epsg:28992", "epsg:4326")

In [12]:
geo = [transformer.transform(l[0], l[1])[:2]  for l in list(df_coor.to_records(index=False))]

In [13]:
max_amount = 22

center_lat = 52.1326
center_lon = 5.2913

m = folium.Map(location=[center_lat, center_lon], zoom_start=7,)

folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)

folium.LayerControl().add_to(m)

In [14]:
points = []
for p in geo:    
    folium.Circle(
        radius=10,
        location=p[:2],
        fill=True,
    ).add_to(m)

m

### Retrieve data from containerized PostgreSQL database# -q option for suppressing output for less clutter

!pip install psycopg2-binary -q

In [ ]:
# -q option for suppressing output for less clutter

!pip install psycopg2-binary -q

In [ ]:
import psycopg2
import pandas.io.sql as sqlio

conn = psycopg2.connect("host=localhost dbname=postgres user=postgres port=5432 password=password")

### Create table and import antenneregister data

In [ ]:
sql = """
drop table if exists antenneregister;
create table if not exists antenneregister (
  x integer,
  y  integer,
  hoofdsoort text,
  sat_code text,
  postcode text,
  woonplaats text,
  gemeente text,
  plaatsing text,
  ingebruikname text,
  objectid bigint,
  hoogte numeric,
  hoofdstraalrichting text,
  frequentie text,
  vermogen numeric,
  veilige_afstand numeric
);
"""

In [ ]:
## conn.rollback() -- in case transaction has failed and is not rolled back.

c = conn.cursor()
c.execute(sql)
conn.commit()
c.close()

Make sure that folder containing antenneregisterdata is working directory

In [33]:
%pwd
%cd ../data
%pwd

/home/ad/work/data-night-13-bridging-notebooks-and-databases/data


/home/ad/work/data-night-13-bridging-notebooks-and-databases/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/ad/work/data-night-13-bridging-notebooks-and-databases/data'

In [34]:
!psql -U postgres -h localhost -d postgres -c "\copy antenneregister from 'antennes-2025-11-10.csv' with delimiter ';' header csv"

COPY 408048


In [35]:
d = sqlio.read_sql_query("select * from antenneregister limit 4", conn)
d

/tmp/ipykernel_29352/3591795323.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  d = sqlio.read_sql_query("select * from antenneregister limit 4", conn)


,x,y,hoofdsoort,sat_code,postcode,woonplaats,gemeente,plaatsing,ingebruikname,objectid,hoogte,hoofdstraalrichting,frequentie,vermogen,veilige_afstand
0,100006,460068,ZENDAMATEURS,ZENDAMATEUR,2394TR,HAZERSWOUDE-RIJNDIJK,Rijnwoude,None,2008-03-03,12519141801,NaN,--,None,NaN,-1.00
1,100016,396255,ZENDAMATEURS,ZENDAMATEUR,4711BA,None,Rucphen,None,2008-03-03,14367874521,NaN,--,None,NaN,-1.00
2,100016,436542,"5G NR, GSM, LTE",5G NR,2922AJ,KRIMPEN AAN DEN IJSSEL,Krimpen aan den IJssel,2007-12-18,2020-07-21,37548511031,20.5,240.0,2160 MHz,38.1,9.24
3,100016,436542,"5G NR, GSM, LTE",5G NR,2922AJ,KRIMPEN AAN DEN IJSSEL,Krimpen aan den IJssel,2007-12-18,2020-07-21,41241845591,20.5,120.0,2160 MHz,38.1,9.24


### Filtering data: all 5G sites higher than 25m meter in Tilburg, ordered by length (desc)

In [37]:
sql = """
select
  *
from antenneregister
where 
  hoogte > 25
  and gemeente = 'Tilburg'
  and sat_code = '5G NR'
order by hoogte desc;
"""

In [38]:
d = sqlio.read_sql_query(sql, conn)
d

/tmp/ipykernel_29352/3431621624.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  d = sqlio.read_sql_query(sql, conn)


,x,y,hoofdsoort,sat_code,postcode,woonplaats,gemeente,plaatsing,ingebruikname,objectid,hoogte,hoofdstraalrichting,frequentie,vermogen,veilige_afstand
0,134187,396299,"5G NR, GSM, LTE",5G NR,5038TC,TILBURG,Tilburg,2012-05-31,2022-12-01,3517218591,47.3,280.0,3700 MHz,49.0,32.41
1,134187,396299,"5G NR, GSM, LTE",5G NR,5038TC,TILBURG,Tilburg,2012-05-31,2022-12-01,9165315531,47.3,170.0,3700 MHz,49.0,32.41
2,134187,396299,"5G NR, GSM, LTE",5G NR,5038TC,TILBURG,Tilburg,2012-05-31,2022-12-01,77827673861,47.3,20.0,3700 MHz,49.0,32.41
3,134187,396299,"5G NR, GSM, LTE",5G NR,5038TC,TILBURG,Tilburg,2012-05-31,2022-12-01,77827673861,47.3,20.0,3700 MHz,49.0,32.41
4,134187,396299,"5G NR, GSM, LTE",5G NR,5038TC,TILBURG,Tilburg,2012-05-31,2022-12-01,9165315531,47.3,170.0,3700 MHz,49.0,32.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,131958,396886,"5G NR, GSM, LTE",5G NR,5037HC,TILBURG,Tilburg,2024-04-22,2024-05-22,21065418563,25.1,200.0,773 MHz,34.0,50.16
1292,131958,396886,"5G NR, GSM, LTE",5G NR,5037HC,TILBURG,Tilburg,2024-04-22,2024-05-22,21065418562,25.1,200.0,2160 MHz,37.9,50.16
1293,135805,398085,5G NR,5G NR,5015BP,None,Tilburg,2024-07-25,2024-08-08,29039164121,25.1,180.0,3500 MHz,46.0,22.94
1294,135805,398085,5G NR,5G NR,5015BP,None,Tilburg,2024-07-25,2024-08-08,59096830651,25.1,60.0,3500 MHz,46.0,22.94
